In [2]:
import csv
from transformers import pipeline

In [6]:
data = "../files/imdb_master.csv"

In [7]:
labels = {"neg": 0, "pos": 1}
texts = []

with open(data, encoding="ISO-8859-1") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if not (row['label'] == "unsup"):
            texts.append((row['review'], labels[row['label']]))

texts = texts[int(len(texts) / 2) :]

In [13]:
texts[0][0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [21]:
model = pipeline("sentiment-analysis")
sentiment = model(texts[0][0])
print(sentiment)

TypeError: 'list' object is not callable